<a href="https://colab.research.google.com/github/CheshireCat12/Deep_learning_challenges/blob/master/PGGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.autograd import Variable

In [0]:
# Image preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, ),
                                                     std=(0.5, ))])


fasion_mnist = datasets.FashionMNIST(root="./data",
                                     train=True,
                                     transform=transform,
                                     download=True)

# Parameters
params_loader = {'batch_size': 32,
                 'shuffle': False}

train_loader = DataLoader(fasion_mnist, **params_loader)

In [0]:
# Generator model

class Generator(nn.Module):
    def __init__(self, Z_dim, ngf, ncc):
        super(Generator, self).__init__()
        
        self.layers = nn.Sequential(
            # input is Z, going into a convolution
            # in_channels, out_channels, kernel_size, stride=1, padding=0
            # formula: (in-1)* stride - 2*padding + kernel_size
            # 128 -> 
            
            # 1 -> (1-1)*1 - 2*0 + 4 = 4
            # in: (100 x 4 x 4)
            nn.ConvTranspose2d(Z_dim, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, ngf, 4, 2, 1, bias=False),
            nn.Tanh(),
            # state size. (ncc) x 64 x 64
            nn.ConvTranspose2d(ngf, ncc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (ncc) x 128 x 128
        )
        
    def forward(self, input_):
        output = self.layers(input_)
        # print(output.shape)
        return output

In [34]:
# Discriminator model

class Discriminator(nn.Module):
    def __init__(self, img_size, ncc=3, init_depth=64, max_depth=1024):
        super(Discriminator, self).__init__()
        
        self.img_size = img_size
        self.ncc = ncc
        self.init_depth = init_depth
        self.max_depth = max_depth
        
        self._create_layers()
        
    def _create_layers(self):
        internal_layers = []
        depth_in = self.ncc
        depth_out = self.init_depth
        
        # Assume img size power of 2!
        for _ in range(1, int(np.log2(self.img_size))-1):
            # Reduce the img size by 2 each iteration
            internal_layers += self._init_block(depth_in, depth_out)
            depth_in = depth_out
            if not depth_out == self.max_depth:
                depth_out *= 2
        
        # Last layer different from the others : depth x 4 x 4 -> 1 x 1 x 1
        internal_layers.append(nn.Conv2d(depth_in, 1, 4, 1, 0, bias=False))
        
        self.depth_in = depth_in
        self.depth_out = depth_out
        self.layers = nn.Sequential(*internal_layers)
        
    def _init_block(self, depth_in, depth_out):
        block = [nn.Conv2d(depth_in, depth_out, 4, 2, 1, bias=False),
                 nn.BatchNorm2d(depth_out),
                 nn.LeakyReLU(0.2)]
        
        return block
    
    def from_rgb(self, dim):
        layer = nn.Conv2d()
    
    def grow_network(self):
        pass
        
        
    
    def forward(self, input_):
        output = self.layers(input_)

        return output.view(-1, 1).squeeze(1)
    
discriminator = Discriminator(4)
print(discriminator)

Discriminator(
  (layers): Sequential(
    (0): Conv2d(3, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)


In [0]:
# https://github.com/pytorch/examples/blob/master/dcgan/main.py
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [0]:
# Dimension of the latent space
Z_dim = 100

# Number of color channel in the final image
ncc = 3

# Number of internal node
ngf, ndf = 64, 64

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")